<a href="https://colab.research.google.com/github/Yiting916/data_analysis/blob/main/(%E5%80%8B%E5%B0%88)combine_pets_and_babies_2019_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 掛載 google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

# 讀取原始 CSV 檔案
df_birth = pd.read_csv('/content/drive/MyDrive/YiTing/Learning/Python/個專/縣市出生嬰兒登記數_全部_2019-2023.csv')
df_pets = pd.read_csv('/content/drive/MyDrive/YiTing/Learning/Python/個專/縣市寵物登記數_貓狗_2019-2023.csv')

# 1. 處理寵物資料：先加總各縣市的貓狗登記數
df_pets_sum = df_pets.groupby(['年', '縣市'])['登記數'].sum().reset_index()
df_pets_sum['寵物類別'] = '全部'

# 2. 計算每年全台的總登記數
df_pets_total = df_pets_sum.groupby('年')['登記數'].sum().reset_index()
df_pets_total['縣市'] = '全台'
df_pets_total['寵物類別'] = '全部'

# 3. 合併縣市資料和全台資料
df_pets_combined = pd.concat([df_pets_sum, df_pets_total])

# 4. 將結果存為新的 CSV 檔案（資料表3）
df_pets_combined.to_csv('/content/drive/MyDrive/YiTing/Learning/Python/個專/縣市寵物登記數_全部_2019-2023.csv', index=False)

# 5. 將寵物資料轉為寬格式以準備合併
df_pets_wide = df_pets_combined.pivot(
    index=['年', '縣市'],
    columns='寵物類別',
    values='登記數'
).reset_index()

# 重命名欄位
df_pets_wide.columns.name = None
df_pets_wide = df_pets_wide.rename(columns={'全部': '寵物登記數'})

# 6. 處理出生資料：移除性別欄位並加總
df_birth_no_gender = df_birth.drop('性別', axis=1).groupby(['年', '縣市'])['嬰兒出生數'].sum().reset_index()

# 7. 合併出生資料和處理後的寵物資料
result = pd.merge(
    df_birth_no_gender,
    df_pets_wide,
    on=['年', '縣市'],
    how='left'
)

# 8. 將最終合併結果存為新的 CSV 檔案
result.to_csv('/content/drive/MyDrive/YiTing/Learning/Python/個專/全台與各縣市的嬰兒出生數與寵物數_合併結果_2019-2023.csv', index=False)